In [ ]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
!pip install pysmiles
from pysmiles import read_smiles
import networkx as nx
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/summaryPubChemAnnotated.csv')


def has_more_chars(string):
  count_numbers = 0
  count_characters = 0
  for character in string:
    if character.isdigit():
      count_numbers += 1
    else:
      count_characters += 1
  return count_characters > count_numbers

def simplify(arr):
  new_arr = arr.copy()

  for i in range(len(arr)):
    try:
      if 'doi' in str(arr[i]).lower():
        new_arr.remove(arr[i])
    except IndexError:
      pass

  for i in range(len(new_arr)):
    new_arr[i] = str(new_arr[i]).replace("<String>", "").replace("</String>", "")

  f_arr = []
  not_allowed = []#'Link to all deposited patent identifiers', 'Patents are available for this chemical structure:', '(The corresponding statement to each P-code can be found at the GHS Classification page.)', 'Subscription Services', 'Legacy Depositors']
  forbidden = []#'©', 'Inc.', 'The GHS information provided by', 'PubMed']
  for i in range(len(new_arr)):
    if (len(new_arr[i].split(' ')) >= 2 or ' is ' in new_arr[i]) and not new_arr[i].count(':') > 3 and not new_arr[i].count('+') > 3 and not any([f in new_arr[i] for f in forbidden]) and new_arr[i] not in not_allowed:
      f_arr.append(new_arr[i])
  if len(str.join(' ', f_arr).split(' ')) > 10:
    return list(set(f_arr))
  else:
    return 'None'

# @limits(calls=5, period=TIME_PERIOD)
def gather_data(cid):
  data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/xml")
  html = BeautifulSoup(data.content, "xml")
  return html.find_all("String")


import time
import pickle
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
Mounted at /content/drive


<ipython-input-1-8aee42ef8d2d>:13: DtypeWarning: Columns (32,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/summaryPubChemAnnotated.csv')


In [ ]:
def scrape_pubchem(df, start=0, end=1):
  df['description'] = np.array([np.nan for i in range(len(df['cid']))])
  st = time.time()
  good = 0
  bad = 0
  for i in range(start, end):
    if good >= 125125:
      break
    if (i+1) % 10 == 0:
      time_diff = (time.time()-st)/60
      pct = round((i-start)/(end-start), 5)
      eta = (time_diff / pct) - time_diff
      clear_output()
      print(f'Finished {(i+1)} out of {(end)}, {round(pct * 100, 2)}% done\nTime gone by: {time_diff // 60} hours and {round(time_diff % 60, 2)} minutes, ETA: {eta // 60} hours and {round(eta % 60, 2)} minutes, Good: {good}, Bad:{bad} Current CID: {df["cid"][i]}')

    if (i+1) % 5000 == 0:
      print('Making checkpoint')
      # pickle.dump(df, open(f'drive/MyDrive/eda_summary_saves/description_checkpoint.v3.{i+1}.pkl', 'wb'))
    x = gather_data(df['cid'][i])

    df['description'][i] = x

  return df

In [ ]:
def gather_data(cid):
  data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/XML?heading=Record+Description")
  html = BeautifulSoup(data.content, "xml")
  x = list(set(html.find_all("String")))
  for i in range(len(x)):
    x[i] = str(x[i]).replace("<String>", "").replace("</String>", "")
  return x

In [ ]:
x = scrape_pubchem(df, start=300000, end=len(df))

Finished 328390 out of 328395, 99.98% done
Time gone by: 1.0 hours and 2.92 minutes, ETA: 0.0 hours and 0.01 minutes, Good: 0, Bad:0 Current CID: 168266236


In [ ]:
pickle.dump(x, open(f'drive/MyDrive/description_full_300k-end_all_sources.pkl', 'wb'))